Get list of publication for given author

In [3]:
import requests

author_id = 'A5113797452'

url = f'https://api.openalex.org/authors/{author_id}/works'

In [16]:
pip install openalexapi

SyntaxError: invalid syntax (463697182.py, line 1)

In [18]:
pip install purl

Note: you may need to restart the kernel to use updated packages.


In [20]:
import openalexapi
import requests, json
import pandas as pd
import ast
import math
import heapq
import csv
import os

name_search = ["david nathan"]


#Getting all author ids through search of first and last name:
base_url = 'https://api.openalex.org/'
def get_authorIDs(name):
    listofIDs = []
    page = 1
    full_query= f'https://api.openalex.org/authors?search={name}&page={page}'
    response = requests.get(full_query)
    visualize_data = response.json()
    num_pages = math.ceil(visualize_data['meta']['count']/25)
    
    while page <= num_pages:
        full_query= f'https://api.openalex.org/authors?search={name}&page={page}'
        response = requests.get(full_query)
        visualize_data = response.json()
        for result in visualize_data['results']:
            openalex_id = result['id'].replace("https://openalex.org/", "")
            listofIDs.append(openalex_id)
            
            #for concepts in result['x_concepts']:
                #if concepts['display_name'] == 'Medicine':
        page += 1 

    print(f'There are {len(listofIDs)} author ids for {name}')
    return listofIDs

#get_authorIDs("William pao")


#Finds all work_ids with given authorId
def work_id(givenAuthorID):
        page = 'page={}'
        filtered_works_url = f'https://api.openalex.org/works?filter=author.id:{givenAuthorID}&{page}'
        page = 1
        has_more_pages = True
        fewer_than_10000_results = True
        all_worksID = []

        # loop through pages
        while has_more_pages and fewer_than_10000_results:

            # set page value and request page from OpenAlex
            url = filtered_works_url.format(page)
            page_with_results = requests.get(url).json()

            # loop through partial list of results
            results = page_with_results['results']
            for i,work in enumerate(results):
                openalex_id = work['id'].replace("https://openalex.org/", "")
                all_worksID.append(openalex_id)
            # next page
            page += 1

            # end loop when either there are no more results on the requested page 
            # or the next request would exceed 15 results
            per_page = page_with_results['meta']['per_page']
            has_more_pages = len(results) == per_page
            fewer_than_10000_results = per_page * page <= 10000
        print(f'There are {len(all_worksID)} works for {givenAuthorID}')
        return (all_worksID)
#work_id('A2250212419')

#Tests if finding asci names and concepts works under a small scale
#----------------------TESTER----------------- FOR THE findAAConcepts FUNCTION AFTER
def findIndvConcepts(names):
    authorsConcepts = {}
    dir = os.path.dirname(os.path.realpath("Open_AlexMerging.ipynb")).replace("open_alex_data", "asci_aap_data")
    os.chdir(dir)
    with open(r"asci_aap_dataJSONUpdated.json") as fileJson:        
        data = json.load(fileJson)
        allData = data["people"]

    for name in names:
        for indv in allData:
            first = indv["first_name"].lower()
            last = indv["last_name"].lower()
            if (first+" "+last)== name.lower():                                               
                authorsConcepts[name] = (ast.literal_eval(indv["original specialization"]))
            
        if (name in authorsConcepts):
            print(f'{name} successfully found')
        else:
            authorsConcepts[name] = []
            print(f'{name} not found') 

    return authorsConcepts
#findIndvConcepts(["William Pao",
                  #"Ashley You",
                  #"Kjersti Aagaard"])
                  #"E. Abel",
                  #"Janis Abkowitz"])


#Goes through asci/aap data and gets name and concepts
def findAAConcepts():
    authorsConcepts = {}
    dir = os.path.dirname(os.path.realpath("Open_AlexMerging.ipynb")).replace("open_alex_data", "asci_aap_data")
    os.chdir(dir)
    with open(r"asci_aap_dataJSONUpdated.json") as fileJson:        
        data = json.load(fileJson)
        allData = data["people"]
        print(f'There are {len(allData)} amount of people in ASCI/AAP json file')

    for indv in allData:
        first = indv["first_name"].lower()
        last = indv["last_name"].lower()
        name = first+" "+last
        if len(indv["original specialization"])!= 2:
            authorsConcepts[name] = (ast.literal_eval(indv["original specialization"]))
    #new_dict = {key: value for key, value in authorsConcepts.items() if value}
    print(f'There are {len(authorsConcepts)} amount of people with specialites listed in ASCI/AAP json file')
    return authorsConcepts  
#findAAConcepts()   

def findAANames():
    authors = []
    dir = os.path.dirname(os.path.realpath("Open_AlexMerging.ipynb")).replace("open_alex_data", "asci_aap_data")
    os.chdir(dir)
    with open(r"asci_aap_dataJSONUpdated.json") as fileJson:        
        data = json.load(fileJson)
        allData = data["people"]
        print(f'There are {len(allData)} amount of people in ASCI/AAP json file')

    for indv in allData:
        first = indv["first_name"].lower()
        last = indv["last_name"].lower()
        name = first+" "+last
        authors.append(name)
    return authors  
#findAANames()

#finished
#finds author concepts given list of names
#filter through medicine 
def authorConcepts(people):
    authors = {}
    for name in people:
        totalConcepts = []
        authorIds = get_authorIDs(name)
        for id in authorIds:
            authorTopics= {}
            tempConcepts = []
            full_query= f'https://api.openalex.org/authors/{id}'
            response = requests.get(full_query)
            visualize_data = response.json()
            for concepts in visualize_data["x_concepts"]:
                if (float(concepts['score']) >= 90.0 and float(concepts['level']) >= 1) or concepts['display_name']== "Medicine":
                    tempConcepts.append(concepts['display_name'])
            authorTopics[id]= tempConcepts
            totalConcepts.append(authorTopics)
        authors[name]= totalConcepts
    return authors 
#authorConcepts(['Kjersti Aagaard'])


#finds all work details given work id
def findWork(workId):
    fullquery = base_url+'works/'+workId
    response = requests.get(fullquery)
    visualize_data = response.json()
    visualize_data.pop("abstract_inverted_index")
    visualize_data.pop("related_works")
    visualize_data.pop("ngrams_url")
    #clean the unicode
    #visualize_data[""]
    return visualize_data
#findWork('W2139236349')


#finds work concepts given work link
def workConcepts(workId):
    totalDict = {}
    totalWorkConcepts = []
    allinfo = findWork(workId)
    for concept in allinfo['concepts']:
        if float(concept['score']) >= 0.3 and float(concept['level']) >= 2 or concept['display_name']== "Medicine" :
            totalWorkConcepts.append(concept['display_name'])
    totalDict[workId] = totalWorkConcepts

    return totalDict
#workConcepts("W2005052157")


#checks which concepts occur the most often in a work
def checkConcepts(conceptlist):
    count_dict = {}
    for element in conceptlist:
        if element!='Medicine':
            if element in count_dict:
                count_dict[element] += 1
            else:
                count_dict[element] = 1

    # Find the three largest values
    largest_values = heapq.nlargest(3, count_dict.values())

    # Find the keys corresponding to the largest values, stores first 3
    final_dict = {}
    n = 0
    for key, value in count_dict.items():
        if n < 3:
            if value in largest_values:
                final_dict[key] = value
                n += 1
        else: 
             break
    return final_dict

testList= ['Medicine',
  'Medicine', 
  'Medicine',
  'Eosinophilic esophagitis',
  'Budesonide',
  'Internal medicine',
  'Heartburn',
  'Eosinophilia',
  'Gastroenterology',
  'Nausea',
  'Vomiting',
  'Corticosteroid',
  'Adverse effect',
  'Esophagitis',
  'Eosinophilic esophagitis',
  'Internal Medicine',
  'Budesonide']
#checkConcepts(testList)

def findWorkConcepts(names): #keys(1. name, 2. authorId 3. workId)
    #searches to get author ids
    finalDict = {}
    count = 0
    for name in names:
        authorIDs = get_authorIDs(name)
        listofAuthors = []
        for id in authorIDs:
            tempDict = {}
            tempWorkList = []
            workIds = work_id(id)
            for wID in workIds:
               if 'Medicine' in workConcepts(wID)[wID]: #preliminary filter
                tempWorkList.append(workConcepts(wID))
               else: 
                   count +=1
            tempDict[id] = tempWorkList
            listofAuthors.append(tempDict)
        finalDict[name] = listofAuthors
    print(f'{count} amount of workIds did not have Medicine in their concepts')
    return finalDict
                
    #searches to get work ids
    #access work ids
    #access concepts in work id
    #loops through concepts in work id and saves it 
#findWorkConcepts(["Seema Aceves"])

PydanticUserError: A non-annotated attribute was detected: `base_url = 'https://api.openalex.org/'`. All model fields require a type annotation; if `base_url` is not meant to be a field, you may be able to resolve this error by annotating it as a `ClassVar` or updating `model_config['ignored_types']`.

For further information visit https://errors.pydantic.dev/2.8/u/model-field-missing-annotation